In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from grid2op.PlotGrid import PlotMatplot
import grid2op
from dataclasses import dataclass


In [7]:
import os
from grid2op import make

# Initialize the env.
env = make("rte_case5_example", test=True)

/home/matthijs/Software/anaconda3/lib/python3.8/site-packages/grid2op/MakeEnv/Make.py:317: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [ ]:
plot_helper = PlotMatplot(env.observation_space)

obs = env.reset()
fig = plot_helper.plot_obs(obs)

In [ ]:
action_space = env.action_space
observation_space = env.observation_space

# Create do_nothing action.
agent_action = action_space({}) # this is the do nothing action
obs, *_ = env.step(agent_action)
print(type(obs))
print(obs.to_dict()) # TODO: does not contain all attributes, e.g. time

In [ ]:
def extract_gen_features(obs: grid2op.Observation.CompleteObservation):
    i = obs.gen_pos_topo_vect
    X = np.array(list(obs.to_dict()['gens'].values())).T
    return i,X

def extract_load_features(obs: grid2op.Observation.CompleteObservation):
    i = obs.load_pos_topo_vect
    X = np.array(list(obs.to_dict()['loads'].values())).T
    return i,X

def extract_or_features(obs: grid2op.Observation.CompleteObservation):
    i = obs.line_or_pos_topo_vect
    X = np.array(list(obs.to_dict()['lines_or'].values())).T
    X = np.concatenate((X,np.reshape(np.array(obs.to_dict()['rho']),(-1,1))),axis=1)
    return i,X

def extract_ex_features(obs: grid2op.Observation.CompleteObservation):
    i = obs.line_ex_pos_topo_vect
    X = np.array(list(obs.to_dict()['lines_ex'].values())).T
    X = np.concatenate((X,np.reshape(np.array(obs.to_dict()['rho']),(-1,1))),axis=1)
    return i,X

print(extract_gen_features(obs))
print(extract_load_features(obs))
print(extract_or_features(obs))
print(extract_ex_features(obs))

In [ ]:
action_space.size()

In [ ]:

agent_action = action_space()
agent_action.change_bus = [1,2]
obs, *_ = env.step(agent_action)
fig = plot_helper.plot_obs(obs)

In [ ]:
obs = env.reset()

In [ ]:
env.action_space.

In [ ]:
load_id = 2
which_connected = mat[obs.load_pos_topo_vect[load_id]] == 1
obs.grid_objects_types[which_connected,]

In [ ]:
@dataclass
class Gen:
    '''
    Dataclass for the static features of a generator
    '''
    id: int
    sub_id: int #Id of the substation
    sub_pos: int #position (RELATIVE) at its substation
    pos: int #position (ABSOLUTE)
        
gens = [Gen(*x) for x in zip(list(range(env.observation_space.n_gen)),
    env.observation_space.gen_to_subid,
    env.observation_space.gen_to_sub_pos,
    env.observation_space.gen_pos_topo_vect)]

@dataclass
class Load:
    '''
    Dataclass for the static features of a load
    '''
    id: int
    sub_id: int #Id of the substation
    sub_pos: int #position (RELATIVE) at its substation
    pos: int #position (ABSOLUTE)
        
loads = [Load(*x) for x in zip(list(range(env.observation_space.n_load)),
    env.observation_space.load_to_subid,
    env.observation_space.load_to_sub_pos,
    env.observation_space.load_pos_topo_vect)]

@dataclass
class PowerLine:
    '''
    Dataclass for the static features of a powerline
    '''
    id: int
    or_sub_id: int #Id of the origin substation
    ex_sub_id: int #Id of the extremity substation
    or_sub_pos: int #Origin position (RELATIVE) at its substation
    ex_sub_pos: int #Extremity position (RELATIVE) at its substation
    or_pos: int #Origin position (ABSOLUTE)
    ex_pos: int #Extremity position (ABSOLUTE)
        
lines = [PowerLine(*x) for x in zip(list(range(env.observation_space.n_line)),
    env.observation_space.line_or_to_subid,
    env.observation_space.line_ex_to_subid,
    env.observation_space.line_or_to_sub_pos,
    env.observation_space.line_ex_to_sub_pos,
    env.observation_space.line_or_pos_topo_vect,
    env.observation_space.line_ex_pos_topo_vect)]